In [3]:
import numpy as np
import pandas as pd

from neo4j import GraphDatabase

In [5]:
import random

transaction_id = [i for i in range(1,50001)]
vendor_number = np.random.randint(low=1, high=2500, size=(50000,))

transaction_amount = np.random.randint(low=20, high=12500000, size=(50000))

transaction_types = ['cash_withdrawal', 'cash_deposit', 'transfer_domestic', 'transfer_international']

random_integers = [random.randint(0,3) for i in range(0,50000)]
transaction_list = [transaction_types[i] for i in random_integers]

transaction_data = {'transaction_id': transaction_id,
                    'vendor_number': list(vendor_number),
                   'transaction_amount': list(transaction_amount),
                   'transaction_type': transaction_list,}

transaction_DataFrame = pd.DataFrame(transaction_data)

In [7]:
transaction_list = transaction_DataFrame.values.tolist()

transaction_execution_commands = []

for i in transaction_list:
    neo4j_create_statement = 'create (t:Transaction {transaction_id:' + str(i[0]) + ', vendor_number: '+ str(i[1]) +'})'
    transaction_execution_commands.append(neo4j_create_statement)

In [13]:
def execute_transactions(transaction_execution_commands):
    data_base_connection = GraphDatabase.driver(uri='bolt://localhost:7687', auth=('neo4j', 'test'))
    session = data_base_connection.session()
    for i in transaction_execution_commands:
        session.run(i)


In [14]:
# print(transaction_execution_commands[:4])
execute_transactions(transaction_execution_commands)